# ఖర్చుల క్లెయిమ్ విశ్లేషణ

ఈ నోట్‌బుక్ లోకల్ రసీదు చిత్రాల నుండి ప్రయాణ ఖర్చులను ప్రాసెస్ చేయడానికి ప్లగిన్లను ఉపయోగించే ఏజెంట్లను ఎలా సృష్టించాలో, ఖర్చుల క్లెయిమ్ ఇమెయిల్‌ను ఎలా రూపొందించాలో, మరియు పై చార్ట్ ఉపయోగించి ఖర్చుల డేటాను ఎలా విజువలైజ్ చేయాలో చూపిస్తుంది. ఏజెంట్లు టాస్క్ కాంటెక్స్ట్ ఆధారంగా ఫంక్షన్లను డైనమిక్‌గా ఎంచుకుంటారు.

దశలు:
1. OCR ఏజెంట్ లోకల్ రసీదు చిత్రాన్ని ప్రాసెస్ చేసి ప్రయాణ ఖర్చుల డేటాను ఎక్స్‌ట్రాక్ట్ చేస్తుంది.
2. ఇమెయిల్ ఏజెంట్ ఖర్చుల క్లెయిమ్ ఇమెయిల్‌ను రూపొందిస్తుంది.

### ప్రయాణ ఖర్చుల సన్నివేశం ఉదాహరణ:
మీరు మరో నగరంలో బిజినెస్ మీటింగ్ కోసం ప్రయాణిస్తున్న ఉద్యోగి అని ఊహించుకోండి. మీ కంపెనీ అన్ని తగిన ప్రయాణ సంబంధిత ఖర్చులను తిరిగి చెల్లించే విధానాన్ని కలిగి ఉంది. ఇక్కడ సాధ్యమైన ప్రయాణ ఖర్చుల విభజన ఉంది:
- రవాణా:
మీ నివాస నగరం నుండి గమ్యస్థాన నగరానికి రౌండ్ ట్రిప్ ఎయిర్‌ఫేర్.
ఎయిర్‌పోర్ట్‌కు మరియు తిరిగి టాక్సీ లేదా రైడ్-హైలింగ్ సేవలు.
గమ్యస్థాన నగరంలో స్థానిక రవాణా (పబ్లిక్ ట్రాన్సిట్, రెంటల్ కార్లు లేదా టాక్సీలు వంటి).

- వసతి:
మీటింగ్ ప్రాంగణానికి సమీపంలోని మధ్యస్థాయి బిజినెస్ హోటల్‌లో మూడు రాత్రుల హోటల్ బస.

- భోజనాలు:
కంపెనీ యొక్క పర్ డియం విధానంపై ఆధారపడి బ్రేక్‌ఫాస్ట్, లంచ్, మరియు డిన్నర్ కోసం రోజువారీ భోజన భత్యం.

- ఇతర ఖర్చులు:
ఎయిర్‌పోర్ట్‌లో పార్కింగ్ ఫీజులు.
హోటల్‌లో ఇంటర్నెట్ యాక్సెస్ ఛార్జీలు.
టిప్స్ లేదా చిన్న సేవా ఛార్జీలు.

- డాక్యుమెంటేషన్:
మీరు అన్ని రసీదులను (ఫ్లైట్స్, టాక్సీలు, హోటల్, భోజనాలు మొదలైనవి) మరియు పూర్తి చేసిన ఖర్చుల నివేదికను తిరిగి చెల్లింపుల కోసం సమర్పిస్తారు.


## అవసరమైన లైబ్రరీలను దిగుమతి చేసుకోండి

నోట్‌బుక్ కోసం అవసరమైన లైబ్రరీలు మరియు మాడ్యూల్‌లను దిగుమతి చేసుకోండి.


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## ఖర్చుల మోడల్స్ నిర్వచించండి

వ్యక్తిగత ఖర్చుల కోసం ఒక Pydantic మోడల్ మరియు వినియోగదారు ప్రశ్నను నిర్మిత ఖర్చుల డేటాగా మార్చడానికి ExpenseFormatter క్లాస్‌ను సృష్టించండి.

ప్రతి ఖర్చు ఈ ఫార్మాట్‌లో ప్రాతినిధ్యం వహిస్తుంది:  
`{'date': '07-Mar-2025', 'description': 'గమ్యస్థానానికి విమానం', 'amount': 675.99, 'category': 'Transportation'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## ఏజెంట్లను నిర్వచించడం - ఇమెయిల్ రూపొందించడం

ఖర్చుల క్లెయిమ్‌ను సమర్పించడానికి ఇమెయిల్ రూపొందించడానికి ఒక ఏజెంట్ క్లాస్‌ను సృష్టించండి.  
- ఈ ఏజెంట్ `kernel_function` డెకరేటర్‌ను ఉపయోగించి ఖర్చుల క్లెయిమ్‌ను సమర్పించడానికి ఇమెయిల్ రూపొందించే ఫంక్షన్‌ను నిర్వచిస్తుంది.  
- ఇది ఖర్చుల మొత్తం మొత్తాన్ని లెక్కించి, వివరాలను ఇమెయిల్ బాడీగా ఫార్మాట్ చేస్తుంది.  


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# రసీదు చిత్రాల నుండి ప్రయాణ ఖర్చులను తీసుకునే ఏజెంట్

రసీదు చిత్రాల నుండి ప్రయాణ ఖర్చులను తీసుకునే ఏజెంట్ క్లాస్‌ను సృష్టించండి.
- ఈ ఏజెంట్ `kernel_function` డెకరేటర్‌ను ఉపయోగించి రసీదు చిత్రాల నుండి ప్రయాణ ఖర్చులను తీసుకునే ఫంక్షన్‌ను నిర్వచిస్తుంది.
- రసీదు చిత్రాన్ని OCR (ఆప్టికల్ క్యారెక్టర్ రికగ్నిషన్) ఉపయోగించి టెక్స్ట్‌గా మార్చి, తేదీ, వివరణ, మొత్తం, మరియు వర్గం వంటి సంబంధిత సమాచారాన్ని తీసుకోండి.


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## ఖర్చులను ప్రాసెస్ చేయడం

ఖర్చులను ప్రాసెస్ చేయడానికి అవసరమైన ఏజెంట్లను సృష్టించి, రిజిస్టర్ చేసి, వాటిని ఆహ్వానించే అసింక్రోనస్ ఫంక్షన్‌ను నిర్వచించండి.  
- ఈ ఫంక్షన్ ఖర్చులను ప్రాసెస్ చేయడం కోసం ఎన్విరాన్‌మెంట్ వేరియబుల్స్‌ను లోడ్ చేసి, అవసరమైన ఏజెంట్లను సృష్టించి, వాటిని ప్లగిన్లుగా రిజిస్టర్ చేస్తుంది.  
- ఇది రెండు ఏజెంట్లతో ఒక గ్రూప్ చాట్‌ను సృష్టించి, ఖర్చుల డేటా ఆధారంగా ఇమెయిల్ మరియు పై చార్ట్‌ను రూపొందించడానికి ప్రాంప్ట్ మెసేజ్‌ను పంపుతుంది.  
- చాట్ ఆహ్వాన సమయంలో ఏవైనా లోపాలు సంభవించినప్పుడు వాటిని నిర్వహించి, ఏజెంట్లను సక్రమంగా క్లీనప్ చేయడం నిర్ధారిస్తుంది.  


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## ప్రధాన ఫంక్షన్

కన్సోల్‌ను క్లియర్ చేసి, `process_expenses` ఫంక్షన్‌ను అసింక్రోనస్‌గా నడపడానికి ప్రధాన ఫంక్షన్‌ను నిర్వచించండి.


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**విమర్శ**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలలో తప్పులు లేదా అసమానతలు ఉండవచ్చు. దాని స్వదేశీ భాషలోని అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించాలి. కీలకమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
